In [8]:
import cv2
import numpy as np
import mediapipe as mp

In [9]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=True,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [10]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks

In [11]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": [landmarks[30], [landmarks[158][0], landmarks[145][1]]],
        "eye_right": [landmarks[287], [landmarks[260][0], landmarks[381][1]]],
        # "shade": [landmarks[71], (landmarks[301][0], landmarks[301][1] + (landmarks[119][1] - landmarks[301][1]))]
        "shade": [(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]]
    }
    return coordinates

In [12]:
def draw_coordinates(image, points, color = [150, 0, 200]):
    neighbor_vector = [(0,0), (1,0), (0,1), (-1,0), (0, -1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for pt in points:
        image[pt[1]][pt[0]] = color
    return image

In [ ]:
def get_angle(coordinates):
    height = coordinates

In [16]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    frame = frame[:,::-1]
    
    if ret:
        landmarks = get_landmarks(frame)
        coordinates = get_coordinates(landmarks)
        image = draw_coordinates(frame, coordinates['shade'])
        
        cv2.imshow("im", image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows()
            cam.release()
            break

KeyError: 30

: 